This produces (x4) **video** for a LR video using a fine-tuned Baseline SRGAN


---


Download this notebook, upload it on google colab and then execute


---


Note that it only works for video frames with number of pixels less than or equal to 76,800, due to OOM error. Hence, if you upload a video of size more than the specified size, it will resize the video and then upsample it


---

Upload the input LR **mp4** video and weights, preferably in the '/content/' folder. You can find the weights in the 'SRGAN-Finetuned' folder of this repo.


---


It is advisable to connect runtime to standard GPU


---


This is a SISR technique, hence it will upsample single image frame at a time. This makes it a bit slower and limits the input size

# Importing Libraries


In [ ]:
import timeit
import math
import cv2 
import os
import numpy as np
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import save_img
from tensorflow.keras.layers import Add, BatchNormalization, Conv2D, Dense, Flatten, Input, PReLU, Lambda
from tensorflow.keras.models import Model

# Defining function to load images

In [ ]:
def load_image(path):
    return np.array(Image.open(path))

# Defining the SRGAN model 

In [ ]:
def resolve(model, LR_batch):
    LR_batch = tf.cast(LR_batch, tf.float32)
    SR_batch = model(LR_batch)
    SR_batch = tf.clip_by_value(SR_batch, 0, 255)
    SR_batch = tf.round(SR_batch)
    SR_batch = tf.cast(SR_batch, tf.uint8)
    return SR_batch

def single_resolve(model, LR):
    return resolve(model, tf.expand_dims(LR, axis=0))[0]

def pixelshuffler(scale):
    return lambda X: tf.nn.depth_to_space(X, scale)

In [ ]:
def normalize_01(X):
    return X / 255.0

def normalize_m11(X):
    return X / 127.5 - 1

def denormalize_m11(X):
    return (X + 1) * 127.5

In [ ]:
def Resnetblock(X_input, num_filters, momentum=0.8):
    X = Conv2D(num_filters, kernel_size=3, padding='same')(X_input)
    X = BatchNormalization(momentum=momentum)(X)
    X = PReLU(shared_axes=[1, 2])(X)
    X = Conv2D(num_filters, kernel_size=3, padding='same')(X)
    X = BatchNormalization(momentum=momentum)(X)
    X = Add()([X_input, X])
    return X

def Upsample(X_input, num_filters):
    X = Conv2D(num_filters, kernel_size=3, padding='same')(X_input)
    X = Lambda(pixelshuffler(scale=2))(X)
    return PReLU(shared_axes=[1, 2])(X)

def Generator(num_filters=64, num_of_resnet_blocks=16):
    X_in = Input(shape=(None, None, 3))
    X = Lambda(normalize_01)(X_in)

    X = Conv2D(num_filters, kernel_size=9, padding='same')(X)
    X = X_1 = PReLU(shared_axes=[1, 2])(X)

    for _ in range(num_of_resnet_blocks):
        X = Resnetblock(X, num_filters)

    X = Conv2D(num_filters, kernel_size=3, padding='same')(X)
    X = BatchNormalization()(X)
    X = Add()([X_1, X])

    X = Upsample(X, num_filters * 4)
    X = Upsample(X, num_filters * 4)

    X = Conv2D(3, kernel_size=9, padding='same', activation='tanh')(X)
    X = Lambda(denormalize_m11)(X)

    return Model(X_in, X)

SRGAN = Generator

# Instantiating the generator
Add the path to the weights file as an argument(str) to the load_weights() function in the following hidden cell

In [ ]:
SRGAN_model = SRGAN()
SRGAN_model.load_weights('/content/ganweight(after 26k epochs).h5') #copy the path of our .h5 weights file and past it here

# Defining the function to apply SRGAN on video


In [ ]:
def video_resizer(orig_vid_path, fps_, h, w): 

  cap = cv2.VideoCapture(orig_vid_path)
  A = 76800
  if (h>w):
    r = (h/w)
    new_w = int(math.sqrt(A/r))
    new_h = int(math.sqrt(A*r))
  else :
    r = (w/h)
    new_h = int(math.sqrt(A/r))
    new_w = int(math.sqrt(A*r))
  
  print("resized height=", new_h, "and resized width=", new_w)
  fourcc = cv2.VideoWriter_fourcc(*'DIVX')
  out = cv2.VideoWriter('/content/input_video_resized.mp4',fourcc, fps_, (new_w,new_h))
  
  while True:
      ret, frame = cap.read()
      if ret == True:
          b = cv2.resize(frame,(new_w,new_h),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
          out.write(b)
      else:
          break
      
  cap.release()
  out.release()

In [ ]:
def SRGAN_on_video(vid_path, kps):

  video = cv2.VideoCapture(vid_path)
  (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

  if int(major_ver)  < 3 :
    fps = int(video.get(cv2.cv.CV_CAP_PROP_FPS))
  else :
    fps = int(video.get(cv2.CAP_PROP_FPS))

  if (fps == 0): 
    print('pls upload a different video')
    quit()

  print(fps)

  hop = round(fps/kps)
  if hop == 0:
    print('pls select such custom fps such that fps<=',fps)
    quit()

  h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  
  print("orig height=", h, "and orig width=", w)
  if (h*w > 76800):
    video_resizer(vid_path, fps, h, w)
    video = cv2.VideoCapture('/content/input_video_resized.mp4')

  arr_output_names = []

  frames_dir = '/content/framesdir'
  os.makedirs(frames_dir)
  output_dir = '/content/outframes'
  os.makedirs(output_dir)

  currentframe = 0       

  while(True):
      ret,frame = video.read()
      if ret:
        if (currentframe % hop == 0):
          name = frames_dir + '/' + str(int(currentframe/hop)) + '.jpg'
          cv2.imwrite(name, frame) 
          LR = load_image(name)
          SR = single_resolve(SRGAN_model, LR)
          img_pil = array_to_img(SR)
          out_name = output_dir + '/' + str(int(currentframe/hop)) + '.jpg'
          img = save_img(out_name, img_pil)
          arr_output_names.append(out_name)
        currentframe += 1
      else:
        break

  video.release()

  image = cv2.imread(output_dir + '/0.jpg')
  height, width, layers = image.shape

  size = (width,height)

  out = cv2.VideoWriter('/content/finetuned_SRGAN_output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), kps , size)

  for i in range(len(arr_output_names)):
    out.write(cv2.imread(arr_output_names[i]))
  out.release()


# Apply the fine-tuned generator on your video
Add the path(str) to your custom video and frame rate per second (int and <30) as arguments to the SRGAN_on_video() function given in the below cell

The output video will be saved as '/content/finetuned_SRGAN_output.mp4' and you can find the input resized video (if any) as '/content/input_video_resized.mp4'

In [ ]:
SRGAN_on_video("/content/CCTV.mp4", 6)

29
orig height= 1080 and orig width= 1920
resized height= 207 and resized width= 369


KeyboardInterrupt: ignored

# Removing the stored input and output frames

In [ ]:
for image in os.listdir('/content/framesdir'):
  os.remove('/content/framesdir' + '/' + image)
os.rmdir('/content/framesdir')
for image in os.listdir('/content/outframes'):
  os.remove('/content/outframes' + '/' + image)
os.rmdir('/content/outframes')

FileNotFoundError: ignored